**1-EXTRACCION Y TRANSFORMACION DE DATOS**
=

1.1-Extracción de los datos
-

In [1139]:
import pandas as pd

movies_original = pd.read_csv('Datos/movies_dataset.csv')
movies = movies_original.copy()

C:\Users\rrangel4\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3258: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


1.2-Eliminación de columnas que no se van a utilizar
-

In [1140]:
# Lista de nombres de columnas que se eliminarán
columns_to_drop = ["video", "imdb_id", "adult", "original_title", "poster_path", "homepage"]

# Eliminar las columnas del DataFrame "data"
movies = movies.drop(columns=columns_to_drop)


1.3-Eliminación de las filas donde el dato contenido del campo "release date" es nulo
-

In [1141]:
print(movies.loc[movies['release_date'].isna()].shape)


(87, 18)


In [1142]:
movies.drop(movies.loc[movies['release_date'].isna()].index, inplace=True)

In [1143]:
print(movies.loc[movies['release_date'].isna()].shape)


(0, 18)


1.4-Imputación de valores para los datos nulos en los campos "revenue" y "budget"
-

In [1144]:
print(movies.loc[movies['revenue'].isna()].shape)
print(movies.loc[movies['budget'].isna()].shape)

(3, 18)
(0, 18)


In [1145]:
values = {"revenue": 0, "budget": 0}
movies.fillna(inplace=True, value=values)

In [1146]:
print(movies.loc[movies['revenue'].isna()].shape)
print(movies.loc[movies['budget'].isna()].shape)

(0, 18)
(0, 18)


1.5-Analisis de valores nulos en otras columnas
-

In [1147]:
movies.isna().sum()

belongs_to_collection    40888
budget                       0
genres                       0
id                           0
original_language           11
overview                   941
popularity                   2
production_companies         0
production_countries         0
release_date                 0
revenue                      0
runtime                    249
spoken_languages             3
status                      83
tagline                  24981
title                        3
vote_average                 3
vote_count                   3
dtype: int64

In [1148]:
# Se observan registros con valor nulo en la columna "title"
print(movies[movies.title.isna()])

      belongs_to_collection                            budget  \
19730              0.065736  /ff9qCepilowshEtG2GYWwzt2bs4.jpg   
29503              1.931659  /zV8bHuSL6WXoD6FWogP9j4x80bL.jpg   
35587              2.185485  /zaSf5OG7V8X8gqFvly88zDdRm46.jpg   

                                                  genres          id  \
19730  [{'name': 'Carousel Productions', 'id': 11176}...  1997-08-20   
29503  [{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...  2012-09-29   
35587  [{'name': 'Odyssey Media', 'id': 17161}, {'nam...  2014-01-01   

      original_language  overview             popularity production_companies  \
19730             104.0  Released                    NaN                False   
29503              68.0  Released                    NaN                False   
35587              82.0  Released  Beware Of Frost Bites                False   

      production_countries release_date  revenue  runtime spoken_languages  \
19730                  6.0            1      0.

In [1149]:
#Se observa que los registros sin titulo en realidad parecen inconsistentes, pues no tienen ningun 
#dato de la columna "runtime" en adelante. Borrado de los registros que no cuentan con título ni estado
movies.drop(movies.loc[movies.title.isna()].index, inplace=True)
movies.isna().sum()

belongs_to_collection    40888
budget                       0
genres                       0
id                           0
original_language           11
overview                   941
popularity                   0
production_companies         0
production_countries         0
release_date                 0
revenue                      0
runtime                    246
spoken_languages             0
status                      80
tagline                  24978
title                        0
vote_average                 0
vote_count                   0
dtype: int64

1.6-Análisis de rango de valores por columna del dataset MOVIES
-

In [1150]:
#Se crea un dataset auxiliar que se usará en algunos analisis
movies_analisys = movies.copy()

In [1151]:
# Columna "belongs_to_collection"
print("Describe de la columna belongs_to_collection: ")
print(movies.belongs_to_collection.describe())


Describe de la columna belongs_to_collection: 
count                                                  4488
unique                                                 1695
top       {'id': 415931, 'name': 'The Bowery Boys', 'pos...
freq                                                     29
Name: belongs_to_collection, dtype: object


In [1152]:
# La columna "belongs_to_collection" es un diccionario. Se resuelve el diccionario extrayendo unicamente el nombre de la franquicia
import ast

movies['franquicia'] = movies['belongs_to_collection'].apply(lambda x: ast.literal_eval(x)['name'] if pd.notna(x) else None)
movies.drop(columns = ['belongs_to_collection'],inplace=True)
movies.rename(columns={"franquicia": "belongs_to_collection"},inplace=True)
movies[['title','belongs_to_collection']][movies['belongs_to_collection'].notna()].head()

,title,belongs_to_collection
0,Toy Story,Toy Story Collection
2,Grumpier Old Men,Grumpy Old Men Collection
4,Father of the Bride Part II,Father of the Bride Collection
9,GoldenEye,James Bond Collection
12,Balto,Balto Collection


In [1153]:
# Identificamos el numero de colecciones y las colecciones que mas peliculas tienen
movies_by_collection = movies[['belongs_to_collection','id']].groupby(['belongs_to_collection']).count()
movies_by_collection
print('Hay',movies_by_collection.id.count(),'colecciones diferentes')
movies_by_collection.sort_values(by='id', ascending = False).head(10)

Hay 1695 colecciones diferentes


,id
belongs_to_collection,
The Bowery Boys,29
Totò Collection,27
James Bond Collection,26
Zatôichi: The Blind Swordsman,26
The Carry On Collection,25
Pokémon Collection,22
Charlie Chan (Sidney Toler) Collection,21
Godzilla (Showa) Collection,16
Uuno Turhapuro,15


In [1154]:
# Columna "budget"
print("Valor minimo y maximo de la columna budget. Min: ",movies.budget.min(), "Max", movies.budget.max(), movies.budget.dtypes)


Valor minimo y maximo de la columna budget. Min:  0 Max 998000 object


In [1155]:
# Los minimos y maximos se estan calculando en sentido alfabetico. Se convierte la variable en numerica
movies['budget'] = movies.budget.astype(int)
print("Valor minimo y maximo de la columna budget. Min: ",movies.budget.min(), "Max", movies.budget.max(), movies.budget.dtypes)

Valor minimo y maximo de la columna budget. Min:  0 Max 380000000 int32


In [1156]:
# Columna "id"
print("Valor minimo y maximo de la columna id. Min: ",movies.id.min(), "Max", movies.id.max(), movies.id.dtypes)

Valor minimo y maximo de la columna id. Min:  100 Max 9999 object


In [1157]:
# Los minimos y maximos se estan calculando en sentido alfabetico. Se convierte la variable en numerica
movies['id'] = movies.id.astype(int)
print("Valor minimo y maximo de la columna id. Min: ",movies.id.min(), "Max", movies.id.max(), movies.id.dtypes)

Valor minimo y maximo de la columna id. Min:  2 Max 469172 int32


In [1158]:
# Varios de los campos internos de las listas de diccionarios se llaman 'id' igual que el campo que es llave primaria del dataset
# Asi que renombramos dicho campo como 'movie_id'
movies.rename(columns={"id": "movie_id"}, inplace=True)


In [1159]:
# Columna "genres"
print("Describe de la columna genres: ")
print(movies.genres.describe())

Describe de la columna genres: 
count                             45376
unique                             4065
top       [{'id': 18, 'name': 'Drama'}]
freq                               4998
Name: genres, dtype: object


In [ ]:
# La columna "genres" es una lista de diccionarios.

In [1162]:
# Columna "original_language"
print("Describe de la columna original_language: ")
print(movies.original_language.describe())
print(movies.original_language.unique())


Describe de la columna original_language: 
count     45365
unique       89
top          en
freq      32202
Name: original_language, dtype: object
['en' 'fr' 'zh' 'it' 'fa' 'nl' 'de' 'cn' 'ar' 'es' 'ru' 'sv' 'ja' 'ko'
 'sr' 'bn' 'he' 'pt' 'wo' 'ro' 'hu' 'cy' 'vi' 'cs' 'da' 'no' 'nb' 'pl'
 'el' 'sh' 'xx' 'mk' 'bo' 'ca' 'fi' 'th' 'sk' 'bs' 'hi' 'tr' 'is' 'ps'
 'ab' 'eo' 'ka' 'mn' 'bm' 'zu' 'uk' 'af' 'la' 'et' 'ku' 'fy' 'lv' 'ta'
 'sl' 'tl' 'ur' 'rw' 'id' 'bg' 'mr' 'lt' 'kk' 'ms' 'sq' nan 'qu' 'te' 'am'
 'jv' 'tg' 'ml' 'hr' 'lo' 'ay' 'kn' 'eu' 'ne' 'pa' 'ky' 'gl' 'uz' 'sm'
 'mt' 'hy' 'iu' 'lb' 'si']


In [1163]:
# Columna "popularity"
print("Describe de la columna popularity: ")
print(movies.popularity.describe())

Describe de la columna popularity: 
count     45376
unique    44142
top       1e-06
freq         30
Name: popularity, dtype: object


In [1169]:
# La variable se va a usar como numerica. Se hace la conversion
movies['popularity'] = movies.popularity.astype(float)
movies_analisys['popularity'] = movies_analisys.popularity.astype(float)

In [1170]:
#Se verifica si la columna popularity contiene outliers: valores muy alejados de la media:
#Para no usar Numpy definimos una funcion que pueda encontra el percentil y asi encontrar los cuartiles 1 y 3
def percentil(per, datos):
    # Copiar la lista para no alterarla
    x = datos.copy()
    x.sort()

    n = len(x)
    rank = (per / 100.0) * (n - 1)
    rank_int = int(rank)
    rank_dec = rank % 1

    if rank_dec:
        p = x[rank_int] + rank_dec * (x[rank_int + 1] - x[rank_int])
    else:
        p = x[rank_int]

    return p

# Usamos la funcion definida para encontrar el Q1 y el Q3
Q1 = percentil(25, movies.popularity.to_list())
Q3 = percentil(75, movies.popularity.to_list())
# Calculamos el rango intercuartil
IQR = Q3-Q1

print ('Popularity: Primer cuartil',Q1, 'Tercer cuartil',Q3, 'Rango intercuartil', IQR)

movies_analisys['popularity_outlayer'] = movies_analisys['popularity'].apply(lambda x: (x < Q1-(1.5*IQR) or x > Q3+(1.5*IQR)))

movies_analisys[['popularity_outlayer','popularity']].groupby(['popularity_outlayer']).count()


Popularity: Primer cuartil 0.3888395 Tercer cuartil 3.6916945 Rango intercuartil 3.302855


,popularity
popularity_outlayer,
False,41168
True,4208


In [ ]:
# Revisando las 4208 peliculas cuya popularidad son outlayers
popularity_analisys = movies_analisys[['title','popularity']][movies_analisys['popularity_outlayer']==True]
popularity_analisys = popularity_analisys.sort_values(by=['popularity'], ascending=False)
print(popularity_analisys)

                      title  popularity
30700               Minions  547.488298
33356          Wonder Woman  294.337037
42222  Beauty and the Beast  287.253654
43644           Baby Driver  228.032744
24455            Big Hero 6  213.849907
...                     ...         ...
15087       Valhalla Rising    8.651468
25981                Tarzan    8.651247
22046                   Ida    8.651053
5677         Half Past Dead    8.650691
34068             Safelight    8.649580

[4208 rows x 2 columns]


In [ ]:
 #Despues de ver los outliers en popularidad, se considera que se deben conservar tal como estan

# Columna "release_date"
print("Describe de la columna release_date: ")
print(movies.release_date.describe())

Describe de la columna release_date: 
count          45376
unique         17333
top       2008-01-01
freq             136
Name: release_date, dtype: object


In [ ]:
# Garantizamos el tipo de dato de la fecha de publicacion
movies['release_date'] = pd.to_datetime(movies['release_date'],format='%Y-%m-%d')
#Extraemos el año de la fecha y generamos la catidad de peliculas por año
movies['release_year'] = pd.DatetimeIndex(movies['release_date']).year
# Generamos un conteo de peliculas por año
release_year_analisys = movies[['id','release_year']].groupby(['release_year']).count()
release_year_analisys = release_year_analisys.sort_values(by=['release_year'], ascending=False)

print(release_year_analisys)


                id
release_year      
2020             1
2018             5
2017           532
2016          1604
2015          1905
...            ...
1888             2
1887             1
1883             1
1878             1
1874             1

[135 rows x 1 columns]


In [ ]:
# Columna "revenue"
print("Describe de la columna revenue: ")
print(movies.revenue.describe())


Describe de la columna revenue: 
count    4.537600e+04
mean     1.123010e+07
std      6.438996e+07
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      2.787965e+09
Name: revenue, dtype: float64


In [ ]:
# Generamos una sumatoria del revenue por año
temp_movies=movies[['revenue','release_year']].groupby(['release_year']).sum()
temp_movies = temp_movies.sort_values(by=['release_year'], ascending=False)

print(temp_movies)

                   revenue
release_year              
2020          0.000000e+00
2018          0.000000e+00
2017          1.507589e+10
2016          3.049788e+10
2015          2.881936e+10
...                    ...
1888          0.000000e+00
1887          0.000000e+00
1883          0.000000e+00
1878          0.000000e+00
1874          0.000000e+00

[135 rows x 1 columns]


In [ ]:
# El revenue por año parece consistente con el numero de peliculas por año

In [ ]:
# Columna "runtime"
print("Describe de la columna runtime: ")
print(movies.runtime.describe())

Describe de la columna runtime: 
count    45130.000000
mean        94.181675
std         38.341059
min          0.000000
25%         85.000000
50%         95.000000
75%        107.000000
max       1256.000000
Name: runtime, dtype: float64


In [ ]:
# Columna "status"
print("Describe de la columna status: ")
print(movies.status.describe())

Describe de la columna status: 
count        45296
unique           6
top       Released
freq         44936
Name: status, dtype: object


In [ ]:
# Generamos la lista unica de estados y su conteo
temp_movies=movies[['id','status']].groupby(['status']).count()
temp_movies = temp_movies.sort_values(by=['status'], ascending=False)

print(temp_movies)

                    id
status                
Rumored            230
Released         44936
Post Production     97
Planned             13
In Production       19
Canceled             1


In [ ]:
# Se eliminan las peliculas en estados diferentes a Released
movies.drop(movies.loc[movies['status']!='Released'].index, inplace=True)
# Hacemos un nuevo conteo por estado
temp_movies=movies[['id','status']].groupby(['status']).count()
temp_movies = temp_movies.sort_values(by=['status'], ascending=False)

print(temp_movies)

             id
status         
Released  44936


In [ ]:
# Columna "title"
print("Describe de la columna title: ")
print(movies.title.describe())

Describe de la columna title: 
count          44936
unique         41813
top       Cinderella
freq              11
Name: title, dtype: object


In [ ]:
# El conteo total es mucho mayor que el conteo de únicos. Generamos un  conteo por titulo
temp_movies=movies[['id','title']].groupby(['title']).count()
temp_movies = temp_movies.sort_values(by=['id'], ascending=False)

print(temp_movies.head(20))

                               id
title                            
Cinderella                     11
Alice in Wonderland             9
Hamlet                          9
Beauty and the Beast            8
Les Misérables                  8
The Three Musketeers            7
A Christmas Carol               7
Treasure Island                 7
Blackout                        7
Heidi                           6
Love                            6
The Hound of the Baskervilles   6
Countdown                       6
Macbeth                         6
Home                            6
The Journey                     6
The Stranger                    6
The Hunters                     6
The Promise                     6
The Circle                      6


In [ ]:
# Analizamos las 3 peliculas mas repetidas por título
movies[movies['title'].isin(['Cinderella','Alice in Wonderland', 'Hamlet'])][['id','title','budget','original_language','popularity','release_date','revenue','runtime','status','vote_average','vote_count']].sort_values(['title','release_date'])


,id,title,budget,original_language,popularity,release_date,revenue,runtime,status,vote_average,vote_count
39530,41278,Alice in Wonderland,0,en,0.708323,1903-10-17,0.000000e+00,9.0,Released,6.1,20.0
32901,87300,Alice in Wonderland,0,en,0.140291,1915-01-15,0.000000e+00,52.0,Released,5.5,2.0
15868,25694,Alice in Wonderland,0,en,1.504355,1933-12-22,0.000000e+00,76.0,Released,5.4,8.0
1003,12092,Alice in Wonderland,3000000,en,16.488937,1951-07-03,5.720000e+08,75.0,Released,7.0,1557.0
26894,35109,Alice in Wonderland,0,en,1.716036,1966-12-26,0.000000e+00,72.0,Released,6.9,8.0
34695,84774,Alice in Wonderland,0,ru,0.320034,1981-01-31,0.000000e+00,30.0,Released,8.7,6.0
11887,34573,Alice in Wonderland,14000000,en,3.794133,1985-12-09,0.000000e+00,187.0,Released,6.3,15.0
9665,30923,Alice in Wonderland,21000000,en,8.614303,1999-02-28,0.000000e+00,150.0,Released,6.1,71.0
14892,12155,Alice in Wonderland,200000000,en,17.285093,2010-03-03,1.025491e+09,108.0,Released,6.4,4773.0
41074,114108,Cinderella,0,fr,1.301063,1899-10-01,0.000000e+00,5.0,Released,5.5,19.0


In [ ]:
# De una misma pelicula se hacen multiples versiones. 

#Hay dos peliculas Hamlet en el año 2000. Generamos una lista por pelicula por año , tratando de identificar si hay peliculas duplicadas
temp_movies=movies[movies['title'].isin(['Cinderella','Alice in Wonderland', 'Hamlet'])][['id','title','release_year']].groupby(['title','release_year']).count()
temp_movies.sort_values('id',ascending=False)

id
title               release_year    
Hamlet              2000           2
Alice in Wonderland 1903           1
                    1915           1
Hamlet              1996           1
                    1990           1
                    1980           1
                    1969           1
                    1964           1
                    1948           1
Cinderella          2015           1
                    2012           1
                    2011           1
                    2002           1
                    2000           1
                    1997           1
                    1957           1
                    1950           1
                    1947           1
                    1914           1
                    1899           1
Alice in Wonderland 2010           1
                    1999           1
                    1985           1
                    1981           1
                    1966           1
                    1951           1
                    1933           1
Hamlet              2009           1

In [ ]:
#Solo Hamlet esta repetida para el año 2000. Revisamos en detalle estas dos lineas
movies.query('release_year  == 2000 & title == "Hamlet"')[['id','title','budget','original_language','popularity','release_date','revenue','runtime','status','vote_average','vote_count','overview']]

,id,title,budget,original_language,popularity,release_date,revenue,runtime,status,vote_average,vote_count,overview
3476,10688,Hamlet,2000000,en,12.896203,2000-05-12,1568749.0,112.0,Released,6.0,35.0,Modern day adaptation of Shakespeare's immorta...
13313,125705,Hamlet,0,en,0.584251,2000-12-10,0.0,178.0,Released,0.0,0.0,"To be or not to be, etc."


In [ ]:
# La pelicula Hamlet con id 125705 no tiene presupuesto, revenue, ni votos...

# Buscando otras peliculas que tengas las mismas caracteristicas
movies.query('budget  == 0 & revenue == 0.0 & vote_average == 0.0 & vote_count == 0.0')[['id','title','budget','original_language','popularity','release_date','revenue','runtime','status','vote_average','vote_count','overview']]


,id,title,budget,original_language,popularity,release_date,revenue,runtime,status,vote_average,vote_count,overview
83,188588,Last Summer in the Hamptons,0,en,0.531159,1995-11-22,0.0,108.0,Released,0.0,0.0,"Filmed entirely on location in East Hampton, L..."
107,96357,Headless Body in Topless Bar,0,en,0.001346,1995-05-20,0.0,110.0,Released,0.0,0.0,An ex-con holds a group of people hostage in a...
126,290157,Jupiter's Wife,0,en,0.001178,1995-01-01,0.0,87.0,Released,0.0,0.0,"Michel Negroponte, a documentary filmmaker, me..."
132,124636,Sonic Outlaws,0,en,0.001838,1995-08-01,0.0,87.0,Released,0.0,0.0,Within days after the release of Negativland's...
137,124639,Target,0,en,0.001205,1995-08-01,0.0,122.0,Released,0.0,0.0,A subtle yet violent commentary on feudal lords.
...,...,...,...,...,...,...,...,...,...,...,...,...
45432,323132,Altar of Fire,0,en,0.000000,1976-01-01,0.0,45.0,Released,0.0,0.0,This film records a 12 day ritual performed by...
45434,325439,The Wonders of Aladdin,0,en,0.093498,1961-10-31,0.0,93.0,Released,0.0,0.0,Young Aladdin (Donald O'Connor) has a series o...
45452,276895,Deep Hearts,0,en,0.011025,1981-01-01,0.0,58.0,Released,0.0,0.0,"Deep Hearts is a film about the Bororo Fulani,..."
45464,227506,Satan Triumphant,0,en,0.003503,1917-10-21,0.0,87.0,Released,0.0,0.0,"In a small town live two brothers, one a minis..."


In [ ]:
# Sucede lo mismo en 2684 peliculas. No es buena idea borrarlas

# Columna "vote_average"
print("Describe de la columna vote_average: ")
print(movies.vote_average.describe())

Describe de la columna vote_average: 
count    44936.000000
mean         5.629380
std          1.907455
min          0.000000
25%          5.000000
50%          6.000000
75%          6.800000
max         10.000000
Name: vote_average, dtype: float64


In [ ]:
#Vote average es un valor entre 0 y 10 presente en la mayoria del dataset

# Columna "vote_count"
print("Describe de la columna vote_count: ")
print(movies.vote_count.describe())

Describe de la columna vote_count: 
count    44936.000000
mean       111.079179
std        494.036416
min          0.000000
25%          3.000000
50%         10.000000
75%         35.000000
max      14075.000000
Name: vote_count, dtype: float64


1.7-Análisis de duplicidad de registros en el dataset MOVIES
-

In [ ]:
#Identificamos los duplicados por id
duplicates_by_id = movies[movies.duplicated(subset='id', keep=False)]
print('Hay', duplicates_by_id.id.count(), 'duplicados por "id"')
duplicates_by_id[['id','title','budget','original_language','popularity','release_date','revenue','runtime','status','vote_average','vote_count','overview']].sort_values(['id', 'release_date'],ascending=[False,False] ).head(6)

Hay 57 duplicados por "id"


,id,title,budget,original_language,popularity,release_date,revenue,runtime,status,vote_average,vote_count,overview
33184,298721,Cemetery of Splendour,980000,th,2.535419,2015-09-02,0.0,122.0,Released,4.4,50.0,"In a hospital, ten soldiers are being treated ..."
40040,298721,Cemetery of Splendour,980000,th,2.535419,2015-09-02,0.0,122.0,Released,4.4,50.0,"In a hospital, ten soldiers are being treated ..."
24164,265189,Force Majeure,0,sv,12.165685,2014-08-15,1359497.0,118.0,Released,6.8,255.0,"While holidaying in the French Alps, a Swedish..."
45265,265189,Force Majeure,0,sv,12.165685,2014-08-15,1359497.0,118.0,Released,6.8,255.0,"While holidaying in the French Alps, a Swedish..."
13603,168538,"Nana, the True Key of Pleasure",0,en,1.276602,1983-06-13,0.0,92.0,Released,4.7,3.0,"In Zola's Paris, an ingenue arrives at a tony ..."
28860,168538,"Nana, the True Key of Pleasure",0,en,1.276602,1983-06-13,0.0,92.0,Released,4.7,3.0,"In Zola's Paris, an ingenue arrives at a tony ..."


In [ ]:
# De los 57 duplcados por Id revisamos cuales estan completamente duplicados
fully_duplicated_ones = movies[movies.duplicated(keep=False)]
print('Hay',fully_duplicated_ones.id.count(),'peliculas totalmente duplicadas')
fully_duplicated_ones[['id','title','budget','original_language','popularity','release_date','revenue','runtime','status','vote_average','vote_count','overview']].sort_values(['id', 'release_date'],ascending=[False,False] ).head(12)

Hay 31 peliculas totalmente duplicadas


,id,title,budget,original_language,popularity,release_date,revenue,runtime,status,vote_average,vote_count,overview
33184,298721,Cemetery of Splendour,980000,th,2.535419,2015-09-02,0.0,122.0,Released,4.4,50.0,"In a hospital, ten soldiers are being treated ..."
40040,298721,Cemetery of Splendour,980000,th,2.535419,2015-09-02,0.0,122.0,Released,4.4,50.0,"In a hospital, ten soldiers are being treated ..."
24164,265189,Force Majeure,0,sv,12.165685,2014-08-15,1359497.0,118.0,Released,6.8,255.0,"While holidaying in the French Alps, a Swedish..."
45265,265189,Force Majeure,0,sv,12.165685,2014-08-15,1359497.0,118.0,Released,6.8,255.0,"While holidaying in the French Alps, a Swedish..."
13603,168538,"Nana, the True Key of Pleasure",0,en,1.276602,1983-06-13,0.0,92.0,Released,4.7,3.0,"In Zola's Paris, an ingenue arrives at a tony ..."
28860,168538,"Nana, the True Key of Pleasure",0,en,1.276602,1983-06-13,0.0,92.0,Released,4.7,3.0,"In Zola's Paris, an ingenue arrives at a tony ..."
13220,159849,Why We Fight: Divide and Conquer,0,en,0.473322,1943-01-01,0.0,57.0,Released,5.0,1.0,"The third film of Frank Capra's 'Why We Fight""..."
35798,159849,Why We Fight: Divide and Conquer,0,en,0.473322,1943-01-01,0.0,57.0,Released,5.0,1.0,"The third film of Frank Capra's 'Why We Fight""..."
19925,152795,The Congress,0,en,8.534039,2013-05-16,455815.0,122.0,Released,6.4,165.0,More than two decades after catapulting to sta...
21854,152795,The Congress,0,en,8.534039,2013-05-16,455815.0,122.0,Released,6.4,165.0,More than two decades after catapulting to sta...


In [ ]:
#Los completamente duplicados se borran dejando solo la primera instancia del registro
movies = movies[~movies.duplicated(keep="first")]

#Identificamos los duplicados por id + titulo + release_date
duplicates_by_id_tit_rd = movies[movies.duplicated(subset=['id','title','release_date'], keep=False)]
print('Hay ',duplicates_by_id_tit_rd.id.count(),'peliculas duplicadas por id, title y release_date')
duplicates_by_id_tit_rd[['id','title','budget','original_language','popularity','release_date','revenue','runtime','status','vote_average','vote_count','overview']].sort_values(['id', 'title', 'release_date'],ascending=[False,False,False] )

Hay  26 peliculas duplicadas por id, title y release_date


,id,title,budget,original_language,popularity,release_date,revenue,runtime,status,vote_average,vote_count,overview
838,132641,Wife,0,ja,0.096079,1953-04-29,0.0,89.0,Released,0.0,0.0,"Ten years into a marriage, the wife is disappo..."
30001,132641,Wife,0,ja,0.619388,1953-04-29,0.0,89.0,Released,0.0,0.0,"Ten years into a marriage, the wife is disappo..."
4356,110428,Camille Claudel 1915,3512454,fr,0.134014,2013-03-13,115860.0,95.0,Released,7.0,20.0,"Winter, 1915. Confined by her family to an asy..."
23534,110428,Camille Claudel 1915,3512454,fr,0.110065,2013-03-13,115860.0,95.0,Released,7.0,20.0,"Winter, 1915. Confined by her family to an asy..."
5710,109962,Rich and Famous,0,en,12.180836,1981-09-23,0.0,115.0,Released,4.9,7.0,Two literary women compete for 20 years: one w...
20899,109962,Rich and Famous,0,en,10.396878,1981-09-23,0.0,115.0,Released,4.9,7.0,Two literary women compete for 20 years: one w...
2564,84198,A Place at the Table,0,en,0.501046,2012-03-22,0.0,84.0,Released,6.9,7.0,"Using personal stories, this powerful document..."
21116,84198,A Place at the Table,0,en,1.673307,2012-03-22,0.0,84.0,Released,6.9,7.0,"Using personal stories, this powerful document..."
11155,77221,Black Gold,40000000,en,6.652197,2011-12-21,5446000.0,130.0,Released,5.9,77.0,"On the Arabian Peninsula in the 1930s, two war..."
20843,77221,Black Gold,40000000,en,6.475665,2011-12-21,5446000.0,130.0,Released,5.9,77.0,"On the Arabian Peninsula in the 1930s, two war..."


In [ ]:
#Se observa en los 26 registros duplicados que algunas campos numericos asociados a la pelicula es lo que difiere entre un 
#registro y otro de la pelicual. Para solucionar las duplicidades, los campos numericos se promedian. Lista de campos: 
#budget,popularity,revenue,runtime,vote_average, vote_count
ids_con_valores_avg = duplicates_by_id_tit_rd[['id', 'title', 'release_date','budget','popularity','revenue','runtime','vote_average', 'vote_count']].groupby(['id', 'title', 'release_date']).mean()
ids_con_valores_avg.sort_values(['id', 'title', 'release_date'],ascending=[False,False,False] )


,,,budget,popularity,revenue,runtime,vote_average,vote_count
id,title,release_date,,,,,,
132641,Wife,1953-04-29,0,0.357734,0.0,89.0,0.0,0.0
110428,Camille Claudel 1915,2013-03-13,3512454,0.122040,115860.0,95.0,7.0,20.0
109962,Rich and Famous,1981-09-23,0,11.288857,0.0,115.0,4.9,7.0
84198,A Place at the Table,2012-03-22,0,1.087176,0.0,84.0,6.9,7.0
77221,Black Gold,2011-12-21,40000000,6.563931,5446000.0,130.0,5.9,77.0
69234,The Phantom of the Opera,1990-03-18,10000000,0.440181,0.0,168.0,5.0,3.0
22649,A Farewell to Arms,1932-12-08,4,2.162944,25.0,89.0,6.2,29.0
15028,Clockstoppers,2002-03-17,26000000,5.146899,38793283.0,94.0,4.9,89.5
14788,Bubble,2005-09-03,1600000,3.096777,0.0,73.0,6.4,36.0


In [ ]:
# Se hace el merge con el dataset original MOVIES para asignar las medias calculadas en los registros "repetidos" que tienen metricas diferentes
dup_movs_valores_avg = movies.merge(ids_con_valores_avg, how='inner', on=['id', 'title', 'release_date'], suffixes=('_orig', '_avg'))
dup_movs_valores_avg[['id', 'title', 'release_date','budget_orig','budget_avg','popularity_orig','popularity_avg','revenue_orig','revenue_avg','runtime_orig','runtime_avg','vote_average_orig','vote_average_avg', 'vote_count_orig', 'vote_count_avg']].head(8)


,id,title,release_date,budget_orig,budget_avg,popularity_orig,popularity_avg,revenue_orig,revenue_avg,runtime_orig,runtime_avg,vote_average_orig,vote_average_avg,vote_count_orig,vote_count_avg
0,132641,Wife,1953-04-29,0,0,0.096079,0.357734,0.0,0.0,89.0,89.0,0.0,0.0,0.0,0.0
1,132641,Wife,1953-04-29,0,0,0.619388,0.357734,0.0,0.0,89.0,89.0,0.0,0.0,0.0,0.0
2,22649,A Farewell to Arms,1932-12-08,4,4,1.914697,2.162944,25.0,25.0,89.0,89.0,6.2,6.2,29.0,29.0
3,22649,A Farewell to Arms,1932-12-08,4,4,2.411191,2.162944,25.0,25.0,89.0,89.0,6.2,6.2,29.0,29.0
4,84198,A Place at the Table,2012-03-22,0,0,0.501046,1.087176,0.0,0.0,84.0,84.0,6.9,6.9,7.0,7.0
5,84198,A Place at the Table,2012-03-22,0,0,1.673307,1.087176,0.0,0.0,84.0,84.0,6.9,6.9,7.0,7.0
6,10991,Pokémon: Spell of the Unknown,2000-07-08,16000000,16000000,10.264597,8.372487,68411275.0,68411275.0,93.0,93.0,6.0,6.0,143.0,143.5
7,10991,Pokémon: Spell of the Unknown,2000-07-08,16000000,16000000,6.480376,8.372487,68411275.0,68411275.0,93.0,93.0,6.0,6.0,144.0,143.5


In [ ]:
#De los duplicados sacamos un unico registro con los valores promedio
unique_movs_valores_avg = dup_movs_valores_avg[dup_movs_valores_avg.duplicated(subset=['id','title','release_date'], keep="first")]

#Del dataset original se eliminan los registros duplicados
movies_sin_dup = movies[~movies.duplicated(subset=['id','title','release_date'], keep=False)]

#se concatenan los dos dataset para obtener el dataset final
dataset_list = [movies_sin_dup, unique_movs_valores_avg]  
movies = pd.concat(dataset_list)


1.8-Generacion de la columna "return" a partir de las columnas "revenue" y "budget"
-

In [1166]:
# Reemplazamos los valores ausentes de la columna "revenue" por ceros
movies['revenue'].fillna(0, inplace = True)

# Reemplazamos los valores ausentes de la columna "budget" por ceros
movies['budget'].fillna(0, inplace = True)

# Calculamos la columna "return"                    
movies['return'] = movies.apply(lambda x: x['revenue']/x['budget'] if not x['budget'] == 0 else 0, axis=1)

movies[['movie_id', 'title', 'budget', 'revenue', 'return']].head()

,movie_id,title,budget,revenue,return
0,862,Toy Story,30000000,373554033.0,12.451801
1,8844,Jumanji,65000000,262797249.0,4.043035
2,15602,Grumpier Old Men,0,0.0,0.000000
3,31357,Waiting to Exhale,16000000,81452156.0,5.090760
4,11862,Father of the Bride Part II,0,76578911.0,0.000000


1.9-Listas de diccionarios
-

1.9.1. Columna "genres"

In [ ]:

# Convertimos la representacion del campo genres tipo string a una lista de diccionarios real
# Esta linea puede fallar si se ejecuta mas de una vez por lo que se incluye el bloque try-except
try:
	movies['genres'] = movies['genres'].apply(ast.literal_eval)
except Exception as varname:
	print('Conversion a lista de diccionarios omitida')


# Aplicamos explode a la columna 'genres' para crear registros separados por cada genero.
# Se aplica en un dataset aparte (exploded movies) para que no cambie la cardinalidad del dataset original
exploded_movies_x_genres = movies.explode('genres')
# En la nueva columna 'genero' se obtiene el nombre del genero
exploded_movies_x_genres['genero'] = exploded_movies_x_genres['genres'].apply(lambda x: x['name'] if pd.notna(x) else None)
exploded_movies_x_genres['id_genero'] = exploded_movies_x_genres['genres'].apply(lambda x: x['id'] if pd.notna(x) else None)
# Obtenemos una lista unica y ordenada de generos
genres_master = exploded_movies_x_genres[['id_genero','genero']].drop_duplicates().dropna().sort_values(by=['id_genero']).reset_index(drop=True)
genres_master['id_genero'] = genres_master['id_genero'].astype('int32')
genres_master.head(3)

1.9.2. Columna "production_companies"

In [1172]:
# Convertimos la representacion del campo production_companies tipo string a una lista de diccionarios real
# Esta linea puede fallar si se ejecuta mas de una vez por lo que se incluye el bloque try-except
try:
	movies['production_companies'] = movies['production_companies'].apply(ast.literal_eval)
except Exception as varname:
	print('Conversion a lista de diccionarios omitida')


# Aplicamos explode a la columna 'production_companies' para crear registros separados por cada productora.
# Se aplica en un dataset aparte (exploded_movies_x_pc) para que no cambie la cardinalidad del dataset original
exploded_movies_x_pc = movies.explode('production_companies')
# En la nueva columna 'productora' se obtiene el nombre de la productora
exploded_movies_x_pc['productora'] = exploded_movies_x_pc['production_companies'].apply(lambda x: x['name'] if pd.notna(x) else None)
exploded_movies_x_pc['id_productora'] = exploded_movies_x_pc['production_companies'].apply(lambda x: x['id'] if pd.notna(x) else None)
# Obtenemos una lista unica y ordenada de productoras
pcs_master = exploded_movies_x_pc[['id_productora','productora']].drop_duplicates().dropna().sort_values(by=['id_productora']).reset_index(drop=True)
pcs_master['id_productora'] = pcs_master['id_productora'].astype('int32')
pcs_master.head(3)

Conversion a lista de diccionarios omitida


,id_productora,productora
0,1,Lucasfilm
1,2,Walt Disney Pictures
2,3,Pixar Animation Studios


1.9.3. Columna "production_countries"

In [1173]:
# Convertimos la representacion del campo production_countries tipo string a una lista de diccionarios real
# Esta linea puede fallar si se ejecuta mas de una vez por lo que se incluye el bloque try-except
try:
	movies['production_countries'] = movies['production_countries'].apply(ast.literal_eval)
except Exception as varname:
	print('Conversion a lista de diccionarios omitida')


# Aplicamos explode a la columna 'production_countries' para crear registros separados por cada pais.
# Se aplica en un dataset aparte (exploded_movies_x_pct) para que no cambie la cardinalidad del dataset original
exploded_movies_x_pct = movies.explode('production_countries')
# En la nueva columna 'pais' se obtiene el nombre del pais
exploded_movies_x_pct['pais'] = exploded_movies_x_pct['production_countries'].apply(lambda x: x['name'] if pd.notna(x) else None)
exploded_movies_x_pct['id_pais'] = exploded_movies_x_pct['production_countries'].apply(lambda x: x['iso_3166_1'] if pd.notna(x) else None)
# Obtenemos una lista unica y ordenada de paises
pcts_master = exploded_movies_x_pct[['id_pais','pais']].drop_duplicates().dropna().sort_values(by=['id_pais']).reset_index(drop=True)
#pcts_master['id_pais'] = pcts_master['id_pais'].astype('int32')
pcts_master.head(3)

,id_pais,pais
0,AE,United Arab Emirates
1,AF,Afghanistan
2,AL,Albania


1.9.4. Columna "spoken_languages"

In [1177]:
# Convertimos la representacion del campo spoken_languages tipo string a una lista de diccionarios real
# Esta linea puede fallar si se ejecuta mas de una vez por lo que se incluye el bloque try-except
try:
	movies['spoken_languages'] = movies['spoken_languages'].apply(ast.literal_eval)
except Exception as varname:
	print('Conversion a lista de diccionarios omitida')


# Aplicamos explode a la columna 'spoken_languages' para crear registros separados por cada idioma.
# Se aplica en un dataset aparte (exploded_movies_x_lan) para que no cambie la cardinalidad del dataset original
exploded_movies_x_lan = movies.explode('spoken_languages')
# En la nueva columna 'idioma' se obtiene el nombre del idioma
exploded_movies_x_lan['idioma'] = exploded_movies_x_lan['spoken_languages'].apply(lambda x: x['name'] if pd.notna(x) else None)
exploded_movies_x_lan['id_idioma'] = exploded_movies_x_lan['spoken_languages'].apply(lambda x: x['iso_639_1'] if pd.notna(x) else None)
# Obtenemos una lista unica y ordenada de idiomas
langs_master = exploded_movies_x_lan[['id_idioma','idioma']].drop_duplicates().dropna().sort_values(by=['id_idioma']).reset_index(drop=True)
#pcts_master['id_pais'] = pcts_master['id_pais'].astype('int32')
langs_master.head(3)

Conversion a lista de diccionarios omitida


,id_idioma,idioma
0,ab,
1,af,Afrikaans
2,am,


1.10 Dataset CREDITS
-

1.10.1 Extracción de los datos
-

In [1179]:
credits_original = pd.read_csv('Datos/credits.csv')
credits = credits_original.copy()
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [1182]:
credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


1.10.2 Listas de diccionarios
-

1.10.2.1 Campo "cast"

In [1187]:
# Convertimos la representacion del campo cast tipo string a una lista de diccionarios real
# Esta linea puede fallar si se ejecuta mas de una vez por lo que se incluye el bloque try-except
try:
	credits['cast'] = credits['cast'].apply(ast.literal_eval)
except Exception as varname:
	print('Conversion a lista de diccionarios omitida')


# Aplicamos explode a la columna 'cast' para crear registros separados por cada persona en el reparto.
# Se aplica en un dataset aparte (exploded_credist_x_cast) para que no cambie la cardinalidad del dataset original
exploded_credist_x_cast = credits.explode('cast')
# En la nueva columna 'actor' se obtiene el nombre de la persona
exploded_credist_x_cast['actor'] = exploded_credist_x_cast['cast'].apply(lambda x: x['name'] if pd.notna(x) else None)
exploded_credist_x_cast['id_actor'] = exploded_credist_x_cast['cast'].apply(lambda x: x['id'] if pd.notna(x) else None)
# Obtenemos una lista unica y ordenada de idiomas
cast_master = exploded_credist_x_cast[['id_actor','actor']].drop_duplicates().dropna().sort_values(by=['id_actor']).reset_index(drop=True)
cast_master['id_actor'] = cast_master['id_actor'].astype('int32')
cast_master.head(4)

Conversion a lista de diccionarios omitida


,id_actor,actor
0,1,George Lucas
1,2,Mark Hamill
2,3,Harrison Ford
3,4,Carrie Fisher


1.10.2.2 Campo "crew"

In [1189]:
# Convertimos la representacion del campo crew tipo string a una lista de diccionarios real
# Esta linea puede fallar si se ejecuta mas de una vez por lo que se incluye el bloque try-except
try:
	credits['crew'] = credits['crew'].apply(ast.literal_eval)
except Exception as varname:
	print('Conversion a lista de diccionarios omitida')


# Aplicamos explode a la columna 'cast' para crear registros separados por cada persona en el reparto.
# Se aplica en un dataset aparte (exploded_credist_x_cast) para que no cambie la cardinalidad del dataset original
exploded_credist_x_crew = credits.explode('crew')
# En la nueva columna 'persona' se obtiene el nombre de la persona acreditada
exploded_credist_x_crew['persona'] = exploded_credist_x_crew['crew'].apply(lambda x: x['name'] if pd.notna(x) else None)
exploded_credist_x_crew['rol_persona'] = exploded_credist_x_crew['crew'].apply(lambda x: x['job'] if pd.notna(x) else None)
exploded_credist_x_crew['id_persona'] = exploded_credist_x_crew['crew'].apply(lambda x: x['credit_id'] if pd.notna(x) else None)
# Obtenemos una lista unica y ordenada de idiomas
crew_master = exploded_credist_x_crew[['id_persona','rol_persona','persona']].drop_duplicates().dropna().sort_values(by=['id_persona']).reset_index(drop=True)
#crew_master['id_persona'] = crew_master['id_persona'].astype('int32')
crew_master.head(4)

Conversion a lista de diccionarios omitida


,id_persona,rol_persona,persona
0,52fe420dc3a36847f800001f,Screenplay,Aki Kaurismäki
1,52fe420dc3a36847f8000025,Director,Aki Kaurismäki
2,52fe420dc3a36847f800003b,Director of Photography,Timo Salminen
3,52fe420dc3a36847f8000041,Editor,Raija Talvio
